# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
import nltk
nltk.download(['punkt','wordnet','averaged_perceptron_tagger'])

import pandas as pd
import numpy as np

import os, re, pickle
from sqlalchemy import create_engine

#for NLP
from nltk.tokenize import word_tokenize as wt
from nltk.stem import WordNetLemmatizer as WNL

#for ML
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
pd.set_option('display.max_columns', 100)


In [10]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse',engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
df['message'][10]

"There's nothing to eat and water, we starving and thirsty."

In [12]:
df.iloc[10,:]

id                                                                       20
message                   There's nothing to eat and water, we starving ...
original                            Bon repo pa gen anyen menm grangou swaf
genre                                                                direct
related                                                                   1
request                                                                   1
offer                                                                     0
aid_related                                                               1
medical_help                                                              1
medical_products                                                          1
search_and_rescue                                                         0
security                                                                  0
military                                                                  0
child_alone 

In [13]:
df.shape

(26216, 40)

In [14]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

### Note:
| Column name | count | null | type |
| --- | --- | --- | --- |
|message                  | 26216 | non-null | object |
|original                 | 10170 | non-null | object |
|genre                    | 26216 | non-null | object |


are string values, the rest are int type

In [17]:
for col in df.columns:
    print("Unique value counts for {} column ----> {}".format(col,df[col].nunique()))
    print("Unique values for {} column :  {} \n".format(col, df[col].unique().tolist() if len(df[col].unique().tolist())<4 else "Too many unique to show" )) 

Unique value counts for id column ----> 26180
Unique values for id column :  Too many unique to show 

Unique value counts for message column ----> 26177
Unique values for message column :  Too many unique to show 

Unique value counts for original column ----> 9630
Unique values for original column :  Too many unique to show 

Unique value counts for genre column ----> 3
Unique values for genre column :  ['direct', 'social', 'news'] 

Unique value counts for related column ----> 3
Unique values for related column :  [1, 0, 2] 

Unique value counts for request column ----> 2
Unique values for request column :  [0, 1] 

Unique value counts for offer column ----> 2
Unique values for offer column :  [0, 1] 

Unique value counts for aid_related column ----> 2
Unique values for aid_related column :  [0, 1] 

Unique value counts for medical_help column ----> 2
Unique values for medical_help column :  [0, 1] 

Unique value counts for medical_products column ----> 2
Unique values for medical_p

In [18]:
df.child_alone.abs().sum()

0

# Note:
Seems like child_alone feature only has 0 value, I am going todrop this column as it has no impact on training process

In [19]:
df.drop('child_alone',axis=1, inplace=True)


In [20]:
df.groupby('related').count()


,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,132,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188


# Note
being related to the disaster topic or nor. this should be binary feature. 2 is probably miss typed, or undefined.
I have 2 option, 
  * Replace 2 with 0 or 1, considering it is misswritten.
  * Drop the rows with related==2 values. which is comparably small proportion of dataset, I believe we will not lose  too much knowledge
  
Removing this values will help us to to get rid of the classification_report 'unknown value type' error caused by different size of related values



In [21]:
df=df[df.related!=2]

In [22]:
df.groupby('related').count()


,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906


In [23]:
X = df['message']
Y = df.iloc[:,4:]

In [24]:
X[:5]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [25]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [26]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
test=dict(df['related'].value_counts())

In [33]:
test

{1: 19906, 0: 6122}

In [64]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = wt(text)
    lemmatizer = WNL()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [65]:
def plumber(clf = AdaBoostClassifier()):

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline


In [35]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', RandomForestClassifier())
    ])

    # specify parameters for grid search
    parameters = {        
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]

    }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [67]:
X_test.shape

(6507,)

In [68]:
model_ada=plumber() #uses AdaBoostClassifier by default for MultiOutputClassifier on top of 
model_ada.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [69]:
Y_pred_ada=model_ada.predict(X_test)

In [70]:
Y.columns.values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related',
       'transport', 'buildings', 'electricity', 'tools', 'hospitals',
       'shops', 'aid_centers', 'other_infrastructure', 'weather_related',
       'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'], dtype=object)

In [71]:

print(classification_report(Y_test.values, Y_pred_ada, target_names=Y.columns.values))


                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4972
               request       0.79      0.54      0.64      1137
                 offer       0.00      0.00      0.00        38
           aid_related       0.78      0.58      0.66      2757
          medical_help       0.60      0.28      0.38       522
      medical_products       0.70      0.31      0.43       344
     search_and_rescue       0.59      0.19      0.29       172
              security       0.17      0.05      0.07       110
              military       0.60      0.37      0.45       210
                 water       0.72      0.60      0.65       419
                  food       0.81      0.71      0.76       734
               shelter       0.80      0.54      0.64       600
              clothing       0.61      0.37      0.46       100
                 money       0.64      0.28      0.39       167
        missing_people       0.56      

### 6. Improve your model
Use grid search to find better parameters. 

In [73]:
model_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f01767138c8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [76]:
model_ada_2=plumber()

parameters = {
    'clf__estimator__learning_rate': [0.1, 0.3, 0.5],
    'clf__estimator__n_estimators': [10, 20, 30]
}

cv = GridSearchCV(model_ada_2, param_grid=parameters, verbose=5, n_jobs=-1)


In [77]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.19317657906869526, total=  29.9s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   37.4s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.19640387275242047, total=  29.9s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.19317657906869526, total=  29.5s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=20 


...

[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=20, score=0.21346242508068233, total=  48.8s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=20 


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  2.8min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=20, score=0.21638235746119563, total=  49.7s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=20, score=0.2128477024742585, total=  49.9s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=30 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=30, score=0.21161825726141079, total= 1.1min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=30 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=30, score=0.2177654833256493, total= 1.1min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=30 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=30, score=0.21177193791301674, total= 1.1min
[CV] clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_est

[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 25.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.3, 0.5], 'clf__estimator__n_estimators': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [78]:
Y_pred_ada_2 = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [79]:
print(classification_report(Y_test.values, Y_pred_ada_2, target_names=Y.columns.values))


                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      4972
               request       0.82      0.46      0.59      1137
                 offer       0.00      0.00      0.00        38
           aid_related       0.79      0.51      0.62      2757
          medical_help       0.69      0.13      0.22       522
      medical_products       0.78      0.22      0.34       344
     search_and_rescue       0.68      0.11      0.19       172
              security       0.00      0.00      0.00       110
              military       0.64      0.17      0.27       210
                 water       0.79      0.56      0.66       419
                  food       0.81      0.72      0.76       734
               shelter       0.82      0.47      0.60       600
              clothing       0.72      0.34      0.46       100
                 money       0.60      0.09      0.16       167
        missing_people       0.75      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [102]:
cv.best_params_

{'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 30}

In [103]:
clf_1=cv.best_estimator_

In [104]:
clf_1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f01767138c8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=0.5, n_estimators=30, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [81]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [87]:
# include StartingVerbExtractor custom transformer
def plumber_startingverb(clf = RandomForestClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

In [88]:
model_rf = plumber_startingverb()
model_rf.fit(X_train, Y_train)
Y_pred_rf = model_rf.predict(X_test)



In [89]:
print(classification_report(Y_test.values, Y_pred_rf, target_names=Y.columns.values))


                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      4972
               request       0.86      0.39      0.54      1137
                 offer       0.00      0.00      0.00        38
           aid_related       0.76      0.52      0.62      2757
          medical_help       0.57      0.06      0.10       522
      medical_products       0.68      0.06      0.10       344
     search_and_rescue       0.60      0.05      0.10       172
              security       0.17      0.01      0.02       110
              military       0.64      0.04      0.08       210
                 water       0.93      0.22      0.36       419
                  food       0.87      0.36      0.51       734
               shelter       0.87      0.14      0.24       600
              clothing       0.76      0.16      0.26       100
                 money       0.83      0.03      0.06       167
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [90]:
parameters_2={'clf__estimator__n_estimators': [50, 100, 200],\
            }
cv_rf_startingverb = GridSearchCV(model_rf, param_grid=parameters_2,n_jobs=-1,verbose=5)

In [91]:
cv_rf_startingverb.fit(X_train,Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.23605348086675887, total= 3.6min
[CV] clf__estimator__n_estimators=50 .................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, score=0.2380513293376364, total= 3.6min
[CV] clf__estimator__n_estimators=50 .................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  8.9min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, score=0.24051021976333178, total= 3.6min
[CV] clf__estimator__n_estimators=100 ................................


...

[CV]  clf__estimator__n_estimators=100, score=0.23651452282157676, total= 6.4min
[CV] clf__estimator__n_estimators=100 ................................


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 20.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, score=0.24619640387275243, total= 6.4min
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.24066390041493776, total= 6.4min
[CV] clf__estimator__n_estimators=200 ................................
[CV]  clf__estimator__n_estimators=200, score=0.23574611956354696, total=11.8min
[CV] clf__estimator__n_estimators=200 ................................
[CV]  clf__estimator__n_estimators=200, score=0.2508068234209313, total=12.0min
[CV] clf__estimator__n_estimators=200 ................................
[CV]  clf__estimator__n_estimators=200, score=0.24235438758260336, total=12.1min


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 76.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [92]:
Y_pred_rf_grid=cv_rf_startingverb.predict(X_test)


In [93]:
print(classification_report(Y_test.values, Y_pred_rf_grid, target_names=Y.columns.values))


                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4972
               request       0.91      0.43      0.58      1137
                 offer       0.00      0.00      0.00        38
           aid_related       0.80      0.62      0.70      2757
          medical_help       0.60      0.04      0.08       522
      medical_products       0.89      0.05      0.09       344
     search_and_rescue       0.67      0.05      0.09       172
              security       0.50      0.01      0.02       110
              military       0.75      0.03      0.06       210
                 water       0.92      0.24      0.38       419
                  food       0.89      0.40      0.55       734
               shelter       0.90      0.24      0.38       600
              clothing       0.86      0.06      0.11       100
                 money       1.00      0.02      0.05       167
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [95]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv_rf_startingverb, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [97]:
cv_rf_startingverb.best_params_

{'clf__estimator__n_estimators': 200}

In [98]:
best_clf=cv_rf_startingverb.best_estimator_

In [100]:
best_clf.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_

In [105]:
with open('best_clf_trained_rf_startverb.pkl', 'wb') as f:
    pickle.dump(best_clf, f)

OSError: [Errno 28] No space left on device

In [107]:
y_pred_last_etaga=best_clf.predict(X_test)

In [108]:
print(classification_report(Y_test.values,y_pred_last_etaga,target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4972
               request       0.91      0.43      0.58      1137
                 offer       0.00      0.00      0.00        38
           aid_related       0.80      0.62      0.70      2757
          medical_help       0.60      0.04      0.08       522
      medical_products       0.89      0.05      0.09       344
     search_and_rescue       0.67      0.05      0.09       172
              security       0.50      0.01      0.02       110
              military       0.75      0.03      0.06       210
                 water       0.92      0.24      0.38       419
                  food       0.89      0.40      0.55       734
               shelter       0.90      0.24      0.38       600
              clothing       0.86      0.06      0.11       100
                 money       1.00      0.02      0.05       167
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
